## RAG System Using Llama2 With Hugging Face

In [5]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 14.7 MB/s eta 0:00:00


In [6]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [7]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [4]:
## Embedding
!pip install git+https://github.com/UKPLab/sentence-transformers.git

  Cloning https://github.com/UKPLab/sentence-transformers.git to /tmp/pip-req-build-cf0cz8gv
  Running command git clone --filter=blob:none --quiet https://github.com/UKPLab/sentence-transformers.git /tmp/pip-req-build-cf0cz8gv
  Resolved https://github.com/UKPLab/sentence-transformers.git to commit 82fa8415b60f85ffb30e8d29353e40bd8fd1bafc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-3.5.0.dev0-py3-none-any.whl size=274786 sha256=fab349497df6341a3ae129abaa64bd27337ac74d98cb619d4b9386dd25bfa720
  Stored in directory: /tmp/pip-ephem-wheel-cache-yb6lisq_/wheels/ed/0d/c8/40f6ac4a13772077a0d91c4fd8cb3f49647f8f857f306edbc9
Successfully built sentence-transformers
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.4.1
    Uninstalling sentence-transformers-3.4.1:
      Successfull

In [8]:
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api
!pip install "transformers[torch]" "huggingface_hub[inference]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00


In [9]:
!pip install llama_index

In [10]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [12]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

Failed to load file /content/data/you-only-look-once.pdf with error: RetryError[<Future at 0x787ac7eddad0 state=finished raised PdfStreamError>]. Skipping...


[Document(id_='67146feb-de40-4813-8b5d-fec13a2b29ea', embedding=None, metadata={'page_label': '1', 'file_name': 'attention-is-all-you-need.pdf', 'file_path': '/content/data/attention-is-all-you-need.pdf', 'file_type': 'application/pdf', 'file_size': 569417, 'creation_date': '2025-02-18', 'last_modified_date': '2025-02-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\

In [13]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `ollama-repo` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ollama

In [15]:
!pip install bitsandbytes

In [19]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Or load_in_8bit=True, depending on the model's quantization
    bnb_4bit_compute_dtype=torch.float16,  # Adjust compute dtype if needed
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    # tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    # model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="deepseek-ai/DeepSeek-R1",
    model_name="deepseek-ai/DeepSeek-R1",
    device_map="auto",
    # model_kwargs={"quantization_config": quantization_config,"torch_dtype": torch.float16 , "load_in_8bit":True}
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

The repository for deepseek-ai/DeepSeek-R1 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/deepseek-ai/DeepSeek-R1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


ValueError: You can't pass `load_in_4bit`or `load_in_8bit` as a kwarg when passing `quantization_config` argument at the same time.

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a90021834c0>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a90021834c0>, id_func=<function default_id_func at 0x7a90eda12320>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7a90021435e0>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a90021834c0>)

In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("what is attention is all you need?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response)

Attention is a powerful tool in NLP, but it is not the only thing you need to build a successful model. While attention mechanisms like the one described in the passage can help the model focus on relevant parts of the input, they do not address other important aspects of language processing, such as syntax and semantics. To build a truly robust NLP model, you will need to incorporate a variety of techniques, including attention, as well as other types of neural network layers and traditional NLP methods.


In [ ]:
response=query_engine.query("what is YOLO?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response)

YOLO is a real-time object detection system that uses a single neural network to predict bounding boxes and class probabilities directly from full images. It is simple, fast, and achieves high performance on object detection tasks. YOLO is trained on full images and directly optimizes detection performance, making it different from traditional object detection methods. It is also shown to be effective in detecting objects in artwork, where other methods struggle.
